In [6]:
import pandas as pd

In [4]:
reviews = pd.read_csv('dataset/review.csv')

KeyboardInterrupt: 

In [5]:
reviews.head()

,funny,user_id,review_id,text,business_id,stars,date,useful,cool
0,0,bv2nCi5Qv5vroFiqKGopiw,v0i_UHJMo_hPBq9bxWvW4w,"Love the staff, love the meat, love the place....",0W4lkclzZThpx3V65bVgig,5,2016-05-28,0,0
1,0,bv2nCi5Qv5vroFiqKGopiw,vkVSCC7xljjrAI4UGfnKEQ,Super simple place but amazing nonetheless. It...,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,0,0
2,0,bv2nCi5Qv5vroFiqKGopiw,n6QzIUObkYshz4dz2QRJTw,Small unassuming place that changes their menu...,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,0,0
3,0,bv2nCi5Qv5vroFiqKGopiw,MV3CcKScW05u5LVfF6ok0g,Lester's is located in a beautiful neighborhoo...,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,0,0
4,0,bv2nCi5Qv5vroFiqKGopiw,IXvOzsEMYtiJI0CARmj77Q,Love coming here. Yes the place always needs t...,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,0,0
